In [4]:
from transformers import pipeline

# Initialize two text generation pipelines
first_generator = pipeline('text-generation', model='distilgpt2')
second_generator = pipeline('text-generation', model='distilgpt2')

def converse(first_model, second_model, num_turns=5):
    conversation = []
    for turn in range(num_turns):
        # First model's turn
        first_turn = first_model("Your prompt here...")
        conversation.append("Model 1: " + first_turn[0]['generated_text'])

        # Second model's turn
        second_turn = second_model("Model 1 says: " + conversation[-1])
        conversation.append("Model 2: " + second_turn[0]['generated_text'])

    return conversation

# Initiate the conversation
conversation = converse(first_generator, second_generator)

# Print the conversation
for utterance in conversation:
    print(utterance)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ValueError: Input length of input_ids is 56, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [2]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
import torch
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
import transformers
import time
import pdb

start0=time.time()
model_dir = './mpt-7b-instruct'

config = AutoConfig.from_pretrained(
  model_dir,
  trust_remote_code=True,
  max_new_tokens=1024
)

with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
    
model.tie_weights()

model = load_checkpoint_and_dispatch(
    model, model_dir, device_map="auto", no_split_module_classes=["MPTBlock"]
)

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
#pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer)

import torch
from transformers import StoppingCriteria, StoppingCriteriaList

# mtp-7b is trained to add "<|endoftext|>" at the end of generations
stop_token_ids = tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model will ramble
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=500,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)
diff0=time.time()-start0
print(diff0, "\n")

start1=time.time()
res=generate_text("Answer the following question:\nQ.My mother has recently been diagnosed with dementia, what support is available for her? \nA.")
print(res[0]["generated_text"])
diff1=time.time()-start1
print(diff1,"\n")

start2=time.time()
res=generate_text("Answer the following question:\nQ.Where can I share my story (about looking after someone with cognitive problems) and hear from others?  \nA.")
print(res[0]["generated_text"])
diff2=time.time()-start2
print(diff2)

pdb.set_trace()

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

configuration_mpt.py:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

fc.py:   0%|          | 0.00/167 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- fc.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


norm.py:   0%|          | 0.00/3.12k [00:00<?, ?B/s]

...

ffn.py:   0%|          | 0.00/5.22k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- ffn.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


blocks.py:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

attention.py:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

ImportError: This modeling file requires the following packages that were not found in your environment: einops. Run `pip install einops`